In [1]:
# Check GPU configuration
!nvidia-smi

Sun Dec 31 04:37:17 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
#@title STEP 0 連接 Google Drive
#@markdown #STEP 0
#@markdown ##連接 Google Drive
#@markdown ##Connect Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#STEP 0 確認有使用GPU
##記事本必須在GPU模式下運行，可以執行上方的nvidia-smi查看GPU是否有GPU。若没有，點擊左上角的 編輯>筆記本設定，把硬體加速器改成GPU。
##如記事本有出問題可在以下儲存庫回報
> https://github.com/ADT109119/Bert-VITS2-Colab

In [2]:
#@title STEP 1 複製儲存庫並安裝必要的函式庫
#@markdown #STEP 1
#@markdown ##複製儲存庫並安裝必要的函式庫
#@markdown ##Clone repository & Install requirements lib

!git clone https://github.com/fishaudio/Bert-VITS2.git
%cd ./Bert-VITS2/
!pip install -r requirements.txt
!pip install pyyaml

#下載 Bert 以及 wavlm 模型
!wget https://huggingface.co/microsoft/wavlm-base-plus/resolve/main/pytorch_model.bin?download=true -O slm/wavlm-base-plus/pytorch_model.bin

!rm -rf bert/chinese-roberta-wwm-ext-large
!git clone https://huggingface.co/hfl/chinese-roberta-wwm-ext-large bert/chinese-roberta-wwm-ext-large

!rm -rf bert/deberta-v2-large-japanese-char-wwm
!git clone https://huggingface.co/ku-nlp/deberta-v2-large-japanese-char-wwm bert/deberta-v2-large-japanese-char-wwm

!rm -rf bert/deberta-v3-large
!git clone https://huggingface.co/microsoft/deberta-v3-large bert/deberta-v3-large


Cloning into 'Bert-VITS2'...
remote: Enumerating objects: 2455, done.
remote: Counting objects: 100% (283/283), done.
remote: Compressing objects: 100% (154/154), done.
remote: Total 2455 (delta 147), reused 219 (delta 115), pack-reused 2172
Receiving objects: 100% (2455/2455), 9.66 MiB | 11.34 MiB/s, done.
Resolving deltas: 100% (1435/1435), done.
/content/Bert-VITS2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.3/214.3 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 751.5/751.5 kB 14.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.0/225.0 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━

# Bert-VITS2 数据预处理

数据准备：
将数据放置在 data 文件夹下，按照如下结构组织：

```
├── data
│   ├── {你的数据集名称}
│   │   ├── esd.list
│   │   ├── raw
│   │   │   ├── ****.wav
│   │   │   ├── ****.wav
│   │   │   ├── ...
```

其中，`raw` 文件夹下保存所有的音频文件，`esd.list` 文件为标签文本，格式为

```
****.wav|{说话人名}|{语言 ID}|{标签文本}\n
```

例如：
```
vo_ABDLQ001_1_paimon_02.wav|派蒙|ZH|没什么没什么，只是平时他总是站在这里，有点奇怪而已
noa_501_0001.wav|NOA|JP|そうだね、油断しないのはとても大事なことだと思う\n"
Albedo_vo_ABDLQ002_4_albedo_01.wav|Albedo|EN|Who are you? Why did you alarm them?
```

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title STEP 2-1 生成上傳按鈕載入資料集
#@markdown #STEP 2-1
#@markdown ##生成上傳按鈕載入資料集
#@markdown ##Upload Dataset

from google.colab import files
import shutil
import os
uploaded = files.upload()
basepath = os.getcwd()
upload_path = "./"
for filename in uploaded.keys():
  shutil.move(os.path.join(basepath, filename), os.path.join(upload_path, "data.zip"))


In [ ]:
#@title STEP 2-2 從Google Drive載入資料集
#@markdown #STEP 2-2
#@markdown ##載入資料集
#@markdown ##Upload Dataset from Google Drive

#@markdown ###資料集路徑
data_set_zip_file = "/content/drive/MyDrive/Bert-VITS2/data.zip" #@param {type:"string"}

!cp -rf {data_set_zip_file} ./data.zip



In [ ]:
#@title STEP 2-3 上傳音檔 透過Whisper生成字幕再自動切分 (因準確度關係 不建議使用)
#@markdown #STEP 2-2
#@markdown ##透過Whisper生成字幕再自動切分
#@markdown ##(因準確度關係 不建議使用)
#@markdown ##Upload audios and generate subtitle by OpenAI Whisper to split audios
#@markdown ##(Not recomand)
#@markdown ##音檔請依照 `{說話者名稱}_{語言}_{隨便的編號}` 格式

!mkdir upload_audios
!mkdir upload_audios/audio
!mkdir upload_audios/srt

upload_from_google_drive = True # @param {type:"boolean"}

if upload_from_google_drive:
  data_folder_path = "/content/drive/MyDrive/Bert-VITS2/audio" #@param {type:"string"}
  !cp -rf {data_folder_path} ./upload_audios/audio
else:
  from google.colab import files
  import shutil
  import os
  uploaded = files.upload()
  basepath = os.getcwd()
  upload_path = "./upload_audios/audio"
  for filename in uploaded.keys():
    shutil.move(os.path.join(basepath, filename), os.path.join(upload_path, filename))

!pip install openai-whisper
import glob

file_list = glob.glob("./upload_audios/audio/*.wav")
for path in file_list:
  !whisper {path} --model large-v3 --output_dir "./upload_audios/srt"

!git clone https://github.com/ADT109119/Bert-VITS2-colab.git ./upload_audios
!python ./upload_audios/processing.py
import shutil
import os

def zip_folder_with_file(folder_path, file_path, output_path):
    with shutil.ZipFile(output_path, 'w') as zipf:
        zipf.write(folder_path, arcname=os.path.basename(folder_path))
        zipf.write(file_path, arcname=os.path.basename(file_path))

zip_folder_with_file('./upload_audios/raw', './upload_audios/esd.list', './data.zip')


In [ ]:
#@title STEP 2.1 解壓縮資料集
#@markdown #STEP 231
#@markdown ##解壓縮資料集
#@markdown ##Unzip Data

#@markdown ###資料集名稱
!mkdir data

data_dir = "dataset" #@param {type:"string"}
!unzip ./data.zip -d ./data/{data_dir}

In [ ]:
#@title STEP 2.5 處理資料
#@markdown #STEP 2.5
#@markdown ##處理資料
#@markdown ##Process Data
from webui_preprocess import generate_config, resample, preprocess_text, bert_gen

batch_size = 8 # @param {type:"integer"}

print(generate_config(data_dir, batch_size))
print(resample(data_dir))
print(preprocess_text(data_dir))
print(bert_gen(data_dir))

#修改 config.yml
import yaml

def load_yaml(file_path):
    with open(file_path, 'r', encoding="utf-8") as file:
        data = yaml.load(file, Loader=yaml.FullLoader)
    return data

def save_yaml(data, file_path):
    with open(file_path, 'w', encoding="utf-8") as file:
        yaml.dump(data, file, default_flow_style=False)

# 載入 YAML 檔案
yaml_file_path = 'config.yml'
yaml_data = load_yaml(yaml_file_path)

# 修改 YAML 中的內容
yaml_data['dataset_path'] = f"data/{data_dir}"
yaml_data['train_ms']['config_path'] = "configs/config.json"

# 儲存修改後的 YAML 檔案
save_yaml(yaml_data, yaml_file_path)


In [ ]:
#@title STEP 2.6 自動儲存步數修改
#@markdown #STEP 2.6
#@markdown ##自動儲存步數修改

#@markdown ###每隔n步自動儲存
save_step = 200 # @param {type:"integer"}

import json

def load_json(file_path):
    with open(file_path, 'r', encoding="utf-8") as file:
        data = json.load(file)
    return data

def save_json(data, file_path):
    with open(file_path, 'w', encoding="utf-8") as file:
        json.dump(data, file, indent=2)

# 載入 JSON 檔案
json_file_path = f"data/{data_dir}/configs/config.json"
json_data = load_json(json_file_path)

# 修改 JSON 中的內容
json_data['train']['log_interval'] = save_step
json_data['train']['eval_interval'] = save_step

# 儲存修改後的 JSON 檔案
save_json(json_data, json_file_path)


# STEP 3 載入模型
## 3-1、3-2 則一

In [ ]:
#@title STEP 3-1 下載預訓練模型
!git clone https://huggingface.co/OedoSoldier/Bert-VITS2-2.3 data/{data_dir}/models

In [ ]:
#@title STEP 3-2 載入上次訓練到一半 儲存在 Google Drive 的模型
!cp -f ../drive/MyDrive/Bert-VITS2/models/* data/{data_dir}/models

#STEP 4 開始訓練

In [ ]:
import os
os.environ['TENSORBOARD_BINARY'] = '/usr/local/bin/tensorboard'
%reload_ext tensorboard
model_dir = f"./data/{data_dir}/models"
%tensorboard --logdir $model_dir

!torchrun --nproc_per_node=1 train_ms.py

#STEP 5 推理測試


In [ ]:
#@title STEP 5 推理測試
#@markdown #STEP 5
#@markdown ##推理測試

#@markdown ###
#@markdown ###推理所使用的模型步數
model_step = 2000 # @param {type:"integer"}
import yaml
def load_yaml(file_path):
    with open(file_path, 'r') as file:
        data = yaml.load(file, Loader=yaml.FullLoader)
    return data
def save_yaml(data, file_path):
    with open(file_path, 'w') as file:
        yaml.dump(data, file, default_flow_style=False)
# 載入 YAML 檔案
yaml_file_path = 'config.yml'
yaml_data = load_yaml(yaml_file_path)
# 修改 YAML 中的內容
yaml_data['dataset_path'] = f"data/{data_dir}"
yaml_data['webui']['share'] = "true"
yaml_data['webui']['model'] = f"models/G_{model_step}.pth"
yaml_data['webui']['config_path'] = "configs/config.json"
# 儲存修改後的 YAML 檔案
save_yaml(yaml_data, yaml_file_path)
!torchrun --nproc_per_node=1 webui.py

#STEP 6 保存到Google Drive(自選)

In [ ]:
#@title STEP 6.1 連接Google Drive
#@markdown ##如果前面連接過了 這邊就不用再連一次
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title STEP 6.2 複製檔案到Google Drive
#@markdown ###模型儲存路徑
Google_Drive_Folder = "/content/drive/MyDrive/Bert-VITS2" #@param {type:"string"}
#@markdown ###儲存模型步數
steps = 2000 # @param {type:"integer"}
!mkdir {Google_Drive_Folder}/models
!cp -rf /content/Bert-VITS2/data/{data_dir}/models/G_{steps}.pth {Google_Drive_Folder}/models/
!cp -rf /content/Bert-VITS2/data/{data_dir}/models/D_{steps}.pth {Google_Drive_Folder}/models/
!cp -rf /content/Bert-VITS2/data/{data_dir}/models/WD_{steps}.pth {Google_Drive_Folder}/models/
!cp -rf /content/Bert-VITS2/data/{data_dir}/models/DUR_{steps}.pth {Google_Drive_Folder}/models/
!cp -rf /content/Bert-VITS2/data/{data_dir}/configs {Google_Drive_Folder}


#STEP 6-2 下載模型

In [ ]:
from google.colab import files
files.download(f"./data/{data_dir}/models/G_{steps}.pth")
files.download(f"./data/{data_dir}/models/D_{steps}.pth")
files.download(f"./data/{data_dir}/models/WD_{steps}.pth")
files.download(f"./data/{data_dir}/models/DUR_{steps}.pth")
files.download(f"./data/{data_dir}/configs/config.json")